In [1]:
from multipie.util.util_binary import BinaryManager
from multipie.util.util_wyckoff import find_wyckoff_site, find_wyckoff_bond
from qtdraw import QtDraw
from multipie.core.multipie_info import __top_dir__
BIN_DIR = __top_dir__ + "others/tools/binary_data"
info = BinaryManager("info", verbose=True, topdir=BIN_DIR)
from utils.util_plot import plot_site, plot_bond, plot_site_samb, plot_bond_samb, plot_vector_samb
qtdraw = QtDraw()
group = BinaryManager("group", topdir=BIN_DIR)
samb_data = BinaryManager("cluster_samb", verbose=True, topdir=BIN_DIR)

load binary from '/Users/hk/Library/CloudStorage/Dropbox/working/GitHub/MultiPie2/others/tools/binary_data/info.pkl'.
"""
* Information of all groups.
- "id_set" (str): (dict) info. of group's IDs.
  - "PG/SG/MPG/MSG" (str):  (dict) ID set.
    - PG: all/crystal/complex/irrep.
    - SG: all/crystal/PG.
    - MPG: all/crystal/PG/type.
    - MSG: all/crystal/PG/SG/MPG/type.
- "tag" (str): (dict) from id to tag.
- "id" (str):  (dict) from tag to id.
- "character" (str): (dict) info. of character.
  - "alias" (str): (dict) alias.
    - "wp/wm" (str): (sympy) definition of wp and wm, exp(2 pi i/3) or exp(-2 pi i/3).
  - "compatibility_relation" (str): (dict) compatibility relation.
    - (PG_tag1,PG_tag2) (str,str): ([(str,str)]) list of compatibility relation, (irrep1,irrep2).
- "root_cluster" (str): (dict) info. of root cluster.
  - "point_group" (str): ([str]) list of PG_tag for root cluster.
  - "rep_site_c" (str): (ndarray(3,sympy)) representative site (fractional for cubic, r=1; = car

In [2]:
tag = "D3^4"
no = info["id"][tag]
gp = group[no]
qtdraw.set_crystal(gp["info"].crystal)
if tag.count("^")==0:
    qtdraw.set_cell("off")
else:
    qtdraw.set_cell("single")
pset = gp["symmetry_operation"]["plus_set"]

samb = samb_data[no]
print(samb["site"].keys())
print(samb["bond_s"].keys())

site = "[1/4,0,1/3]"
bond = "[1/4,0,1/3];[0,1/4,2/3]"
s_wp, sites = find_wyckoff_site(gp, site)
b_wp, bonds = find_wyckoff_bond(gp, bond)
#s_wp = "6l"
#b_wp = "6b@6l"

site_wyckoff = gp["wyckoff"]["site"][s_wp]
# sites = site_wyckoff["reference"].astype(float)
site_mapping = site_wyckoff["mapping"]
bond_wyckoff = gp["wyckoff"]["bond"][b_wp]
# bonds = bond_wyckoff["reference"].astype(float)
bond_mapping = bond_wyckoff["mapping"]

print("s_wp =", s_wp)
print("sites =", sites.tolist())
print("site mapping =", site_mapping)
print("b_wp =", b_wp)
print("bonds =", bonds.tolist())
print("bond mapping =", bond_mapping)

dict_keys(['3a', '3b', '6c'])
dict_keys(['3a@3a', '3b@3a', '6c@3a', '3a@3b', '3b@3b', '6c@3b', '6a@6c'])
s_wp = 3a
sites = [[0.25, 0.0, 0.3333333333333333], [0.0, 0.25, 0.6666666666666666], [0.75, 0.75, 0.0]]
site mapping = [[1, 5], [2, 4], [3, 6]]
b_wp = 3a@3b
bonds = [[-0.25, -0.5, 0.33333333, 0.875, 0.0, 0.8333333333333334], [0.5, 0.25, 0.33333333, 0.0, 0.875, 0.16666666666666666], [-0.25, 0.25, 0.33333333, 0.125, 0.125, 0.5]]
bond mapping = [[1, -5], [2, -4], [3, -6]]


In [3]:
# plot definition of site and bond cluster.
qtdraw.clear_data()
qtdraw.add_text2d(f"site = {s_wp}, bond = {b_wp}")
plot_site(qtdraw, sites)
plot_bond(qtdraw, bonds)

In [4]:
# site-cluster SAMB.
for no, (idx, val) in enumerate(samb["site"][s_wp].items()):
    print(no, ":", idx, val[0].tolist())
s_idx = list(samb["site"][s_wp].keys())

0 : ('Q', 0, 'A1', -1, -1, 0, 0, 'q') [[sqrt(3)/3, sqrt(3)/3, sqrt(3)/3]]
1 : ('Q', 1, 'E', -1, -1, 0, 0, 'q') [[sqrt(6)/3, -sqrt(6)/6, -sqrt(6)/6], [0, sqrt(2)/2, -sqrt(2)/2]]


In [5]:
# plot site-cluster SAMB.
idx = s_idx[1]
comp = 0
qtdraw.clear_data()
qtdraw.add_text2d(f"idx = ({",".join(map(str,idx[:4]))},{comp})")
plot_site_samb(qtdraw, sites, samb["site"][s_wp][idx][0][comp])

In [6]:
# bond-cluster SAMB (symmetric bond).
for no, (idx, val) in enumerate(samb["bond_s"][b_wp].items()):
    print(no, ":", idx, val[0].tolist())
bs_idx = list(samb["bond_s"][b_wp].keys())

0 : ('Q', 0, 'A1', -1, -1, 0, 0, 'q') [[sqrt(3)/3, sqrt(3)/3, sqrt(3)/3]]
1 : ('Q', 1, 'E', -1, -1, 0, 0, 'q') [[sqrt(6)/3, -sqrt(6)/6, -sqrt(6)/6], [0, sqrt(2)/2, -sqrt(2)/2]]


In [7]:
# plot bond-cluster SAMB (symmetric bond).
idx = bs_idx[1]
comp = 0
qtdraw.clear_data()
qtdraw.add_text2d(f"idx = ({",".join(map(str,idx[:4]))},{comp})")
plot_site(qtdraw, sites, rep=False)
plot_bond_samb(qtdraw, bonds, samb["bond_s"][b_wp][idx][0][comp])

In [8]:
# bond-cluster SAMB (anti-symmetric bond).
for no, (idx, val) in enumerate(samb["bond_a"][b_wp].items()):
    print(no, ":", idx, val[0].tolist())
ba_idx = list(samb["bond_a"][b_wp].keys())

0 : ('Q', 1, 'A2', -1, -1, 0, 0, 'q') [[sqrt(3)/3, sqrt(3)/3, sqrt(3)/3]]
1 : ('Q', 1, 'E', -1, -1, 0, 0, 'q') [[0, -sqrt(2)/2, sqrt(2)/2], [sqrt(6)/3, -sqrt(6)/6, -sqrt(6)/6]]


In [9]:
# plot bond-cluster SAMB (anti-symmetric bond).
idx = ba_idx[1]
comp = 0
qtdraw.clear_data()
qtdraw.add_text2d(f"idx = ({",".join(map(str,idx[:4]))},{comp})")
plot_site(qtdraw, sites, rep=False)
plot_bond_samb(qtdraw, bonds, samb["bond_a"][b_wp][idx][0][comp], sym=False)

In [10]:
# bond-cluster vector SAMB.
for no, (idx, val) in enumerate(samb["vector"][b_wp].items()):
    print(no, ":", idx, val[0].tolist())
v_idx = list(samb["vector"][b_wp].keys())

0 : ('Q', 0, 'A1', -1, -1, 1, 1, 'q') [[[sqrt(3)/3, 0, 0], [-sqrt(3)/6, 1/2, 0], [-sqrt(3)/6, -1/2, 0]]]
1 : ('Q', 1, 'A2', -1, -1, 1, -1, 'q') [[[0, 0, sqrt(3)/3], [0, 0, sqrt(3)/3], [0, 0, sqrt(3)/3]]]
2 : ('G', 1, 'A2', -1, -1, 1, 0, 'q') [[[0, -sqrt(3)/3, 0], [1/2, sqrt(3)/6, 0], [-1/2, sqrt(3)/6, 0]]]
3 : ('Q', 1, 'E', -1, -1, 1, -1, 'q') [[[sqrt(3)/3, 0, 0], [sqrt(3)/3, 0, 0], [sqrt(3)/3, 0, 0]], [[0, sqrt(3)/3, 0], [0, sqrt(3)/3, 0], [0, sqrt(3)/3, 0]]]
4 : ('G', 1, 'E', -1, -1, 1, 0, 'q') [[[0, 0, 0], [0, 0, -sqrt(2)/2], [0, 0, sqrt(2)/2]], [[0, 0, sqrt(6)/3], [0, 0, -sqrt(6)/6], [0, 0, -sqrt(6)/6]]]
5 : ('Q', 1, 'E', -1, -1, 1, 1, 'q') [[[sqrt(3)/3, 0, 0], [-sqrt(3)/6, -1/2, 0], [-sqrt(3)/6, 1/2, 0]], [[0, -sqrt(3)/3, 0], [-1/2, sqrt(3)/6, 0], [1/2, sqrt(3)/6, 0]]]


In [11]:
# plot bond-cluster vector SAMB.
idx = v_idx[2]
comp = 0
qtdraw.clear_data()
qtdraw.add_text2d(f"idx = ({",".join(map(str,idx[:4]))},{idx[5]},{comp})")
plot_vector_samb(qtdraw, bonds, samb["vector"][b_wp][idx][0][comp])